In [1]:
import sys
import random
from time import time

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.optim as optim

from KGAT import KGAT
from utils.parser_kgat import *
from utils.log_helper import *
#from utils.metrics import *
from utils.model_helper import *
from data_loader import DataLoaderKGAT

from scipy.sparse import coo_matrix
import copy


# Original code
def evaluate(model, dataloader, user_ids, Ks, device):
    test_batch_size = len(user_ids) #dataloader.test_batch_size
    train_user_dict = dataloader.train_user_dict
    test_user_dict = dataloader.test_user_dict

    model.eval()

    #user_ids = list(test_user_dict.keys())
    user_ids_batches = [user_ids[i: i + test_batch_size] for i in range(0, len(user_ids), test_batch_size)]
    user_ids_batches = [torch.LongTensor(d) for d in user_ids_batches]

    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)

    cf_scores = []
    metric_names = ['precision', 'recall', 'ndcg']
    metrics_dict = {k: {m: [] for m in metric_names} for k in Ks}

    with tqdm(total=len(user_ids_batches), desc='Evaluating Iteration') as pbar:
        for batch_user_ids in user_ids_batches:
            batch_user_ids = batch_user_ids.to(device)

            with torch.no_grad():
                batch_scores = model(batch_user_ids, item_ids, mode='predict')       # (n_batch_users, n_items)

            batch_scores = batch_scores.cpu()
            batch_metrics = calc_metrics_at_k(batch_scores, train_user_dict, test_user_dict, batch_user_ids.cpu().numpy(), item_ids.cpu().numpy(), Ks)

            cf_scores.append(batch_scores.numpy())
            for k in Ks:
                for m in metric_names:
                    metrics_dict[k][m].append(batch_metrics[k][m])
            pbar.update(1)

    cf_scores = np.concatenate(cf_scores, axis=0)
    for k in Ks:
        for m in metric_names:
            metrics_dict[k][m] = np.concatenate(metrics_dict[k][m]).mean()
    return cf_scores, metrics_dict

def predict(args):
    # GPU / CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # load data
    data = DataLoaderKGAT(args, logging)

    # load model
    model = KGAT(args, data.n_users, data.n_entities, data.n_relations)
    checkpoint = torch.load(args.pretrain_model_path, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    model.to(device)

    # predict
    Ks = eval(args.Ks)
    k_min = min(Ks)
    k_max = max(Ks)

    cf_scores, metrics_dict = evaluate(model, data, Ks, device)
    np.save(args.save_dir + 'cf_scores.npy', cf_scores)
    print('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
        metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))

In [2]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss


def calc_recall(rank, ground_truth, k):
    """
    calculate recall of one example
    """
    return len(set(rank[:k]) & set(ground_truth)) / float(len(set(ground_truth)))


def precision_at_k(hit, k):
    """
    calculate Precision@k
    hit: list, element is binary (0 / 1)
    """
    hit = np.asarray(hit)[:k]
    return np.mean(hit)


def precision_at_k_batch(hits, k):
    """
    calculate Precision@k
    hits: array, element is binary (0 / 1), 2-dim
    """
    res = hits[:, :k].mean(axis=1)
    return res


def average_precision(hit, cut):
    """
    calculate average precision (area under PR curve)
    hit: list, element is binary (0 / 1)
    """
    hit = np.asarray(hit)
    precisions = [precision_at_k(hit, k + 1) for k in range(cut) if len(hit) >= k]
    if not precisions:
        return 0.
    return np.sum(precisions) / float(min(cut, np.sum(hit)))


def dcg_at_k(rel, k):
    """
    calculate discounted cumulative gain (dcg)
    rel: list, element is positive real values, can be binary
    """
    rel = np.asfarray(rel)[:k]
    dcg = np.sum((2 ** rel - 1) / np.log2(np.arange(2, rel.size + 2)))
    return dcg


def ndcg_at_k(rel, k):
    """
    calculate normalized discounted cumulative gain (ndcg)
    rel: list, element is positive real values, can be binary
    """
    idcg = dcg_at_k(sorted(rel, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(rel, k) / idcg


def ndcg_at_k_batch(hits, k):
    """
    calculate NDCG@k
    hits: array, element is binary (0 / 1), 2-dim
    """
    hits_k = hits[:, :k]
    dcg = np.sum((2 ** hits_k - 1) / np.log2(np.arange(2, k + 2)), axis=1)

    sorted_hits_k = np.flip(np.sort(hits), axis=1)[:, :k]
    idcg = np.sum((2 ** sorted_hits_k - 1) / np.log2(np.arange(2, k + 2)), axis=1)

    idcg[idcg == 0] = np.inf
    ndcg = (dcg / idcg)
    return ndcg


def recall_at_k(hit, k, all_pos_num):
    """
    calculate Recall@k
    hit: list, element is binary (0 / 1)
    """
    hit = np.asfarray(hit)[:k]
    return np.sum(hit) / all_pos_num


def recall_at_k_batch(hits, k):
    """
    calculate Recall@k
    hits: array, element is binary (0 / 1), 2-dim
    """
    res = (hits[:, :k].sum(axis=1) / hits.sum(axis=1))
    return res


def F1(pre, rec):
    if pre + rec > 0:
        return (2.0 * pre * rec) / (pre + rec)
    else:
        return 0.


def calc_auc(ground_truth, prediction):
    try:
        res = roc_auc_score(y_true=ground_truth, y_score=prediction)
    except Exception:
        res = 0.
    return res


def logloss(ground_truth, prediction):
    logloss = log_loss(np.asarray(ground_truth), np.asarray(prediction))
    return logloss


def calc_metrics_at_k(cf_scores, train_user_dict, test_user_dict, user_ids, item_ids, Ks):
    """
    cf_scores: (n_users, n_items)
    """
    test_pos_item_binary = np.zeros([len(user_ids), len(item_ids)], dtype=np.float32)
    for idx, u in enumerate(user_ids):
        train_pos_item_list = train_user_dict[u]
        test_pos_item_list = test_user_dict[u]
        cf_scores[idx][train_pos_item_list] = -np.inf
        test_pos_item_binary[idx][test_pos_item_list] = 1

    try:
        _, rank_indices = torch.sort(cf_scores.cuda(), descending=True)    # try to speed up the sorting process
    except:
        _, rank_indices = torch.sort(cf_scores, descending=True)
    rank_indices = rank_indices.cpu()

    binary_hit = []
    for i in range(len(user_ids)):
        binary_hit.append(test_pos_item_binary[i][rank_indices[i]])
    binary_hit = np.array(binary_hit, dtype=np.float32)
    metrics_dict = {}
    for k in Ks:
        metrics_dict[k] = {}
        metrics_dict[k]['precision'] = precision_at_k_batch(binary_hit, k)
        metrics_dict[k]['recall']    = recall_at_k_batch(binary_hit, k)
        metrics_dict[k]['ndcg']      = ndcg_at_k_batch(binary_hit, k)
    return metrics_dict


def exp_calc_metrics_at_k(rank_indices, train_user_dict, test_user_dict, user_ids, item_ids, Ks):
    test_pos_item_binary = np.zeros([len(user_ids), len(item_ids)], dtype=np.float32)
    for idx, u in enumerate(user_ids):
        train_pos_item_list = train_user_dict[u]
        test_pos_item_list = test_user_dict[u]
        test_pos_item_binary[idx][test_pos_item_list] = 1

    rank_indices = torch.LongTensor([rank_indices])
    binary_hit = []
    for i in range(len(user_ids)):
        binary_hit.append(test_pos_item_binary[i][rank_indices[i]])
    binary_hit = np.array(binary_hit, dtype=np.float32)

    metrics_dict = {}
    for k in Ks:
        metrics_dict[k] = {}
        metrics_dict[k]['precision'] = precision_at_k_batch(binary_hit, k)
        metrics_dict[k]['recall']    = recall_at_k_batch(binary_hit, k)
        metrics_dict[k]['ndcg']      = ndcg_at_k_batch(binary_hit, k)
    return metrics_dict



In [3]:
# kgat args (can adjust Ks and pretrain_model_path)
class KGAT_args():
    def __init__(self, 
                 seed=2019,
                 data_name='amazon-book', 
                 data_dir='datasets/', 
                 use_pretrain=1, 
                 pretrain_embedding_dir='datasets/pretrain/', 
                 pretrain_model_path='trained_model/KGAT/model_epoch280.pth', 
                 cf_batch_size=1024, 
                 kg_batch_size=2048, 
                 test_batch_size=10000, 
                 embed_dim=64, 
                 relation_dim=64, 
                 laplacian_type='random-walk', 
                 aggregation_type='bi-interaction', 
                 conv_dim_list='[64, 32, 16]', 
                 mess_dropout='[0.1, 0.1, 0.1]', 
                 kg_l2loss_lambda=1e-5, 
                 cf_l2loss_lambda=1e-5, 
                 lr=0.0001, 
                 n_epoch=1000, 
                 stopping_steps=10, 
                 cf_print_every=1, 
                 kg_print_every=1, 
                 evaluate_every=20, 
                 Ks='[10]'):
        
        self.seed = seed
        self.data_name = data_name
        self.data_dir = data_dir
        self.use_pretrain = use_pretrain
        self.pretrain_embedding_dir = pretrain_embedding_dir
        self.pretrain_model_path = pretrain_model_path
        self.cf_batch_size = cf_batch_size
        self.kg_batch_size = kg_batch_size
        self.test_batch_size = test_batch_size
        self.embed_dim = embed_dim
        self.relation_dim = relation_dim
        self.laplacian_type = laplacian_type
        self.aggregation_type = aggregation_type
        self.conv_dim_list = conv_dim_list
        self.mess_dropout = mess_dropout
        self.kg_l2loss_lambda = kg_l2loss_lambda
        self.cf_l2loss_lambda = cf_l2loss_lambda
        self.lr = lr
        self.n_epoch = n_epoch
        self.stopping_steps = stopping_steps
        self.cf_print_every = cf_print_every
        self.kg_print_every = kg_print_every
        self.evaluate_every = evaluate_every
        self.Ks = Ks
        save_dir = 'trained_model/KGAT/{}/embed-dim{}_relation-dim{}_{}_{}_{}_lr{}_pretrain{}/'.format(
        self.data_name, self.embed_dim, self.relation_dim, self.laplacian_type, self.aggregation_type,
        '-'.join([str(i) for i in eval(self.conv_dim_list)]), self.lr, self.use_pretrain)
        self.save_dir = save_dir

In [4]:
args = KGAT_args()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load data
data = DataLoaderKGAT(args, logging)

# load model
model = KGAT(args, data.n_users, data.n_entities, data.n_relations)
checkpoint = torch.load(args.pretrain_model_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
model.to(device)
pass

d:\Benz\study\大四上\FinTech\final\FinTech-Final\data_loader.py:291: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()


In [ ]:
# Counts
print(data.n_items) # 0~24914
print(data.n_entities) # 24915~113486
print(data.n_users_entities) # 113487~184165

In [5]:
def find_all_path(start, end, target_r_1, target_r_2, adj_matrix):
    adj_first = adj_matrix[0].tocsr()
    adj_second = adj_matrix[target_r_1].tocsr()
    #adj_last = adj_matrix[target_r_2].tocsr()
    
    if target_r_2 == 0:
        reverse_adj_last = adj_matrix[1].tocsr()
    elif target_r_2 <= 40:
        reverse_adj_last = adj_matrix[target_r_2 + 39].tocsr()
    else:
        reverse_adj_last = adj_matrix[target_r_2 - 39].tocsr()
    
    # All items connect to start
    first_step_candidate = adj_first.indices[adj_first.indptr[start]:adj_first.indptr[start+1]]
    first_step_candidate = np.delete(first_step_candidate, np.where(first_step_candidate == end)[0])
    
    # All entity(uiei)/user(uiui) connect to end
    last_step_candidate = reverse_adj_last.indices[reverse_adj_last.indptr[end]:reverse_adj_last.indptr[end+1]]
    last_step_candidate = np.delete(last_step_candidate, np.where(last_step_candidate == start)[0])

    output_paths = []
    for h in first_step_candidate:
        tails = adj_second.indices[adj_second.indptr[h]:adj_second.indptr[h+1]]
        for t in tails:
            if t in last_step_candidate:
                output_paths.append([start, h, target_r_1, t, target_r_2, end])
    
    return output_paths

def find_all_path_all_r(start, end, relations, adj_matrix):
    all_paths = []
    for r in relations:
        if r == 1:
            r_1 = 1
            r_2 = 0
        else:
            r_1 = r
            r_2 = r + 39
        
        #t1 = time.perf_counter()
        paths = find_all_path(start, end, r_1, r_2, adj_matrix)
        all_paths += paths
        #t2 = time.perf_counter()
        #print("t = ", t2 - t1)
        paths = find_all_path(start, end, r_2, r_1, adj_matrix)
        all_paths += paths
        #t3 = time.perf_counter()
        #print("t2 = ", t3 - t2)
    return all_paths

In [18]:
# Get the score of a single path 
# 'sum', 'mul', 'max'
def path_score(path, A_in, mode='sum'):
    assert len(path) == 6  # [U, I, r_1, E, r_2, I]

    if mode == 'sum':
        return A_in[path[0]][path[1]] + A_in[path[1]][path[3]] + A_in[path[3]][path[5]]
    
    elif mode == 'mul':
        return A_in[path[0]][path[1]] * A_in[path[1]][path[3]] * A_in[path[3]][path[5]]
    
    elif mode == 'max':
        return max(A_in[path[0]][path[1]], A_in[path[1]][path[3]], A_in[path[3]][path[5]])
    
    elif mode == '12':
        return A_in[path[0]][path[1]] * A_in[path[1]][path[3]]
    
    elif mode == '23':
        return A_in[path[1]][path[3]] * A_in[path[3]][path[5]]

    elif mode == '13':
        return A_in[path[0]][path[1]] * A_in[path[3]][path[5]]

    else:
        raise ValueError('mode should be in ["sum", "mul", "max"].')

# Rerank the paths
def get_top_k_path(paths, A_in, mode='sum', k=5):
    assert len(paths) > 0
    
    scores = [path_score(path, A_in, mode) for path in paths]
    reranked_paths = [path for _, path in sorted(zip(scores,paths), reverse=True)]
    sorted_scores = sorted(scores, reverse=True)

    return reranked_paths[:k], sorted_scores[:k]

# Single user prediction
def predict_single(model, dataloader, user_id, relations, Ks, device, mode):
    train_user_dict = dataloader.train_user_dict

    model.eval()
    input_user_id = torch.LongTensor([user_id]).to(device)

    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)
    k_max = max(Ks)

    with torch.no_grad():
        matching_scores = model(input_user_id, item_ids, mode='predict')[0].cpu().numpy()       # (n_batch_users, n_items)

    train_pos_item_list = train_user_dict[user_id]
    matching_scores[train_pos_item_list] = -np.inf
    top_k_items = np.argsort(-matching_scores)[:k_max]
    top_k_all_att_scores = []
    top_k_all_paths = []
    top_k_paths = []
    top_k_att_scores = []
    
    for item_id in top_k_items:
        paths = find_all_path_all_r(user_id, item_id, relations, dataloader.adjacency_dict)
        reranked_paths, scores = get_top_k_path(paths, model.A_in, mode=mode, k=5)
        top_k_all_att_scores.append(scores)
        top_k_all_paths.append(reranked_paths)
        top_k_att_scores.append(scores[0])
        top_k_paths.append(reranked_paths[0])
    
    output = {}
    for k in Ks:
        reranked_items = [id for _, id in sorted(zip(top_k_att_scores[:k], top_k_items[:k]), reverse=True)]
        explainations = [path for _, path in sorted(zip(top_k_att_scores[:k], top_k_paths[:k]), reverse=True)]
        output[k] = {'item_ids': reranked_items, 'explainations': explainations}
    
    #print(top_k_items)
    #print(reranked_items)
    #print(dataloader.test_user_dict[user_id])
    return output, top_k_all_att_scores, top_k_all_paths, top_k_items

# Check original model predcition hit the GT in top-k (the result is in hits_10.txt, hits_20.txt)
def predict_ori(model, dataloader, Ks, device):
    test_batch_size = 3000 #dataloader.test_batch_size
    train_user_dict = dataloader.train_user_dict
    test_user_dict = dataloader.test_user_dict
    
    model.eval()
    user_ids = list(test_user_dict.keys())
    user_ids_batches = [user_ids[i: i + test_batch_size] for i in range(0, len(user_ids), test_batch_size)]
    user_ids_batches = [torch.LongTensor(d) for d in user_ids_batches]

    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)
    k_max = max(Ks)

    hit_users = []
    with tqdm(total=len(user_ids_batches), desc='Evaluating Iteration') as pbar:
        for batch_user_ids in user_ids_batches:
            batch_user_ids = batch_user_ids.to(device)

            with torch.no_grad():
                matching_scores = model(batch_user_ids, item_ids, mode='predict')       # (n_batch_users, n_items)

            batch_user_ids = batch_user_ids.cpu().numpy()
            test_pos_item_binary = np.zeros([len(batch_user_ids), len(item_ids.cpu().numpy())], dtype=np.float32)
            for idx, u in enumerate(batch_user_ids):
                train_pos_item_list = train_user_dict[u]
                test_pos_item_list = test_user_dict[u]
                matching_scores[idx][train_pos_item_list] = -np.inf
                test_pos_item_binary[idx][test_pos_item_list] = 1

            try:
                _, rank_indices = torch.sort(matching_scores.cuda(), descending=True)    # try to speed up the sorting process
            except:
                _, rank_indices = torch.sort(matching_scores, descending=True)

            rank_indices = rank_indices.cpu()
            binary_hit = []
            for i in range(len(batch_user_ids)):
                binary_hit.append(test_pos_item_binary[i][rank_indices[i]])
            binary_hit = np.array(binary_hit, dtype=np.float32)[:, :k_max]
            hit_user = np.where(np.sum(binary_hit, axis=1) >= 1)[0]
            hit_users += list(batch_user_ids[hit_user])
            pbar.update(1)
    
    return hit_users

# Get results of all users (explainable version)
def explainable_evaluate(model, dataloader, user_ids, relations, Ks, device, mode):
    test_batch_size = dataloader.test_batch_size
    train_user_dict = dataloader.train_user_dict
    test_user_dict = dataloader.test_user_dict

    model.eval()

    #user_ids = list(test_user_dict.keys())
    
    n_items = dataloader.n_items
    item_ids = torch.arange(n_items, dtype=torch.long).to(device)

    metric_names = ['precision', 'recall', 'ndcg']
    metrics_dict = {k: {m: [] for m in metric_names} for k in Ks}

    with tqdm(total=len(user_ids), desc='Evaluating Iteration') as pbar:
        for user_id in user_ids:

            output, _, _, ori_top_k = predict_single(model, dataloader, user_id, relations, Ks, device, mode)
            batch_metrics = exp_calc_metrics_at_k(output[max(Ks)]['item_ids'], train_user_dict, test_user_dict, [user_id], item_ids.cpu().numpy(), Ks)
            ori_metrics = exp_calc_metrics_at_k(ori_top_k, train_user_dict, test_user_dict, [user_id], item_ids.cpu().numpy(), Ks)
            for k in Ks:
                for m in metric_names:
                    metrics_dict[k][m].append(batch_metrics[k][m])
            pbar.update(1)

    for k in Ks:
        for m in metric_names:
            metrics_dict[k][m] = np.concatenate(metrics_dict[k][m]).mean()
    return metrics_dict
    
def explainable_recommend(model, data, relations, Ks):
    # predict
    #Ks = eval(Ks)
    k_min = min(Ks)
    k_max = max(Ks)
    metrics_dicts_sum = []
    metrics_dicts_mul = []
    metrics_dicts_max = []
    metrics_dicts = []
    with open('hits_10.txt', 'r') as f:
        all_hits = f.readlines()
    with open('results/Ks10_5_400.txt', 'w') as f:
        pass
    for i in range(len(all_hits)):
        all_hits[i] = int(all_hits[i])
    for i in range(5):
        user_ids = random.sample(all_hits, 400)
        cf_scores, metrics_dict = evaluate(model, data, user_ids, Ks, device)
        metrics_dicts.append(metrics_dict)
        print("Original")
        print('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        with open('results/Ks10_5_400.txt', 'a') as f:
            f.write("Original\n")
            f.write('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        metrics_dict = explainable_evaluate(model, data, user_ids, relations, Ks, device, mode='sum')
        metrics_dicts_sum.append(metrics_dict)
        print("Explainable sum")
        print('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        with open('results/Ks10_5_400.txt', 'a') as f:
            f.write("Explainable sum\n")
            f.write('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        metrics_dict = explainable_evaluate(model, data, user_ids, relations, Ks, device, mode='mul')
        metrics_dicts_mul.append(metrics_dict)
        print("Explainable mul")
        print('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        with open('results/Ks10_5_400.txt', 'a') as f:
            f.write("Explainable mul\n")
            f.write('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        metrics_dict = explainable_evaluate(model, data, user_ids, relations, Ks, device, mode='max')
        metrics_dicts_max.append(metrics_dict)
        with open('results/Ks10_5_400.txt', 'a') as f:
            f.write("Explainable max\n")
            f.write('CF Evaluation: Precision [{:.4f}, {:.4f}], Recall [{:.4f}, {:.4f}], NDCG [{:.4f}, {:.4f}]'.format(
           metrics_dict[k_min]['precision'], metrics_dict[k_max]['precision'], metrics_dict[k_min]['recall'], metrics_dict[k_max]['recall'], metrics_dict[k_min]['ndcg'], metrics_dict[k_max]['ndcg']))
        
    return metrics_dicts, metrics_dicts_sum, metrics_dicts_mul, metrics_dicts_max

In [61]:
np.where(np.sum(np.array([[0, 0, 0, 0, 0]]), axis=1) > 0)

(array([], dtype=int64),)

In [9]:
#delete [2, 3, 6, 11, 21, 22, 32, 35, 37] -> [4, 5, 8, 13, 23, 24, 34, 37, 39] ([43, 44, 47, 52, 62, 63, 73, 76, 78])
all_relations = range(2, 41)
delete_relations = [4, 5, 8, 13, 23, 24, 34, 37, 39]
target_relations = list(set(all_relations) - set(delete_relations))

Ks = [10]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#output, all_paths, all_scores, ori_top_k = predict_single(model, data, 113511, target_relations, [20], device)
#hits = predict_ori(model, data, [20], device)

In [106]:
with open('hits_20.txt', 'w') as f:
    for hit in hits:
        f.write(str(hit)+'\n')

In [89]:
print(list(ori_top_k))
print(output[20]['item_ids'])
print(data.test_user_dict[113511])

[805, 4693, 324, 1473, 548, 271, 2249, 291, 537, 2841, 642, 2142, 4016, 1078, 6897, 5401, 2040, 534, 6891, 4025]
[6897, 6891, 5401, 4693, 4025, 4016, 2841, 2249, 2142, 2040, 1473, 1078, 805, 642, 548, 537, 534, 324, 291, 271]
[  805 15038]


In [16]:
dicts = explainable_recommend(model, data, target_relations, Ks)

Evaluating Iteration: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


Original
CF Evaluation: Precision [0.1270, 0.1270], Recall [0.6123, 0.6123], NDCG [0.3798, 0.3798]


Evaluating Iteration:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Benz\AppData\Local\Temp/ipykernel_24320/559897981.py:155: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  rank_indices = torch.LongTensor([rank_indices])
Evaluating Iteration:   1%|          | 1/100 [00:15<26:21, 15.98s/it]

[ 1220 13815  3461 16248 14322 18412 16374 20344  8904 15746]
[14322, 18412, 20344, 16374, 16248, 15746, 13815, 8904, 3461, 1220]
[1220]
{10: {'precision': array([0.1], dtype=float32), 'recall': array([1.], dtype=float32), 'ndcg': array([0.28906483])}}
{10: {'precision': array([0.1], dtype=float32), 'recall': array([1.], dtype=float32), 'ndcg': array([1.])}}


Evaluating Iteration:   2%|▏         | 2/100 [00:32<26:15, 16.07s/it]

...

[ 9621  1214 17673  1216 19706  6408 16245  8907 20019 15883]
[6408, 20019, 15883, 16245, 8907, 1214, 19706, 17673, 9621, 1216]
[8270 8907]
{10: {'precision': array([0.1], dtype=float32), 'recall': array([1.], dtype=float32), 'ndcg': array([0.38685281])}}
{10: {'precision': array([0.1], dtype=float32), 'recall': array([1.], dtype=float32), 'ndcg': array([0.31546488])}}


Evaluating Iteration:  54%|█████▍    | 54/100 [16:41<12:55, 16.86s/it]

[ 9100  3404  8073 17800  3403  8074  1440 11105  3402 11426]
[11105, 11426, 9100, 3404, 17800, 8074, 8073, 3403, 1440, 3402]
[11426 15515]
{10: {'precision': array([0.1], dtype=float32), 'recall': array([1.], dtype=float32), 'ndcg': array([0.63092975])}}
{10: {'precision': array([0.1], dtype=float32), 'recall': array([1.], dtype=float32), 'ndcg': array([0.28906483])}}


Evaluating Iteration:  54%|█████▍    | 54/100 [16:47<14:18, 18.66s/it]


KeyboardInterrupt: 